当构建模型时使用在预测时不可用的信息时，就会发生数据泄漏。这会导致性能估计过于乐观，例如 交叉验证 ，因此当模型用于实际新颖的数据时（例如在生产期间）性能较差。

一个常见的原因是没有将测试和训练数据子集分开。永远不要使用测试数据来选择模型。一般规则是永远不要调用`fit`测试数据。虽然这听起来很明显，但在某些情况下很容易错过，例如在应用某些预处理步骤时。

尽管训练和测试数据子集都应该接受相同的预处理转换（如上一节所述），但重要的是这些转换只能从训练数据中学习。例如，如果您有一个除以平均值的标准化步骤，则平均值应该是训练子集的平均值，而不是所有数据的平均值。如果测试子集包含在平均计算中，则来自测试子集的信息会影响模型。

下面详细介绍预处理期间数据泄漏的示例。

# 预处理过程中的数据泄露

我们在这里选择通过特征选择步骤来说明数据泄漏。然而，这种泄漏风险与 scikit-learn 中的几乎所有转换相关，包括（但不限于）StandardScaler、SimpleImputer和PCA.

scikit-learn 中提供了许多 特征选择 功能。它们可以帮助删除不相关、冗余和嘈杂的特征，并改善您的模型构建时间和性能。与任何其他类型的预处理一样，特征选择应该只使用训练数据。在特征选择中包含测试数据将使您的模型产生乐观的偏差。

为了演示，我们将使用 10,000 个随机生成的特征创建这个二元分类问题：

In [1]:
import numpy as np

n_samples, n_features, n_classes = 200, 10000, 2
rng = np.random.RandomState(42)
X = rng.standard_normal((n_samples, n_features))
y = rng.choice(n_classes, n_samples)

**错误的**

即使我们的目标是完全随机的，使用所有数据执行特征选择的准确度得分也远高于偶然性。这种随机性意味着我们的X和y是独立的，因此我们期望准确度在 0.5 左右。但是，由于特征选择步骤“看到”了测试数据，因此模型具有不公平的优势。在下面的错误示例中，**我们首先使用所有数据进行特征选择，然后将数据拆分为训练和测试子集以进行模型拟合**。结果远高于预期的准确度分数：

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [3]:
X_selected = SelectKBest(k=25).fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, random_state=42)

gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)
accuracy_score(y_test, y_pred)

0.76

**正确的**

为防止泄露数据，最好先将数据拆分为训练子集和测试子集。然后可以仅使用训练集形成特征选择。请注意，每当我们使用fit或者fit_transform时，我们只使用训练数据集。分数是我们对数据的预期：

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

select = SelectKBest(k=25)
X_train_selected = select.fit_transform(X_train, y_train)
gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train_selected, y_train)
X_test_selected = select.transform(X_test)
y_pred = gbc.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.46

**使用管道**

在这里，我们再次建议使用pipeline将特征选择和估计器链接在一起。管道确保在执行时仅使用训练数据fit，而测试数据仅用于计算准确度：

In [5]:
from sklearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline = make_pipeline(
    SelectKBest(k=25),
    GradientBoostingClassifier(random_state=1)
)
pipeline.fit(X_train, y_train)
accuracy_score(y_test, pipeline.predict(X_test))

0.46

管道也可以输入到交叉验证函数中，例如cross_val_score。同样，管道确保在拟合和预测期间使用正确的数据子集和估计方法：

In [7]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y)
scores.mean(), scores.std()

(0.45499999999999996, 0.0714142842854285)

# 如何避免数据泄露

- 始终首先将数据拆分为训练和测试子集，尤其是在任何预处理步骤之前
- 使用fit和fit_transform方法时 切勿包含测试数据。使用所有数据会导致过于乐观的分数
- 相反，transform方法应该用于训练和测试子集，因为应该对所有数据应用相同的预处理。这可以通过fit_transform在训练子集和transform在测试子集上使用来实现。
- scikit-learn管道是防止数据泄露的好方法。因为它确保在正确的数据子集上执行适当的方法。该管道非常适用于交叉验证和超参数调整功能。